In [ ]:
import os

# Asegurarse que el directorio de trabajo es la raíz del proyecto
import os
os.chdir(r'D:\ITMeet\Operaciones\BP010-data-pipelines-auditoria')
print(f'Working directory: {os.getcwd()}')
    
print(f"📍 Current working directory: {os.getcwd()}")

In [ ]:
import requests
import sqlalchemy
from sqlalchemy import create_engine, text, pool, insert
from dotenv import load_dotenv
from datetime import datetime, timedelta

import logging

# Configuración básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

load_dotenv()

In [ ]:
def execute_sql(engine, esquema_stage_sql):
    """
    Ejecuta el DDL de Stage para crear todas las tablas.
    """
    try:
        with engine.connect() as connection:

            connection.execute(text(esquema_stage_sql))
            
            connection.commit()
        logger.info("consulta ejecutada exitosamente.")
    except Exception as e:
        logger.error(f"Error al crear tablas de Stage: {e}")
        raise

In [ ]:
# Es buena práctica usar variables de entorno para las credenciales
db_user = os.getenv("DB_USER")
db_name = os.getenv("DB_NAME")

# --- PARÁMETROS DEL TÚNEL SSH ---
db_host = os.getenv("DB_HOST") # Apunta a máquina local
db_port = os.getenv("DB_PORT")        # El puerto local del túnel

In [ ]:
db_password = os.getenv("PRD_DB_PASSWORD")
#db_password = os.getenv("DEV_DB_PASSWORD")

In [ ]:
# 1. Construir la cadena de conexión
STAGE_DATABASE_URL = (f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
print(f"Stage Database URL: {STAGE_DATABASE_URL}")

In [ ]:
def last_data_api_extraction():
    unit_id = os.environ.get("UNIT_ID")
    location_id = os.environ.get("LOCATION_ID")

    url = "https://gx6b35ycyc.execute-api.us-east-1.amazonaws.com/dev/api"

    headers = {
        'x-user': 'ia-hydrog',
        'x-pass': '14@HydR06!$$87tgdguij123AA',
        'Content-Type': 'application/json'}


    get_latest_data_url = f"{url}/get-latest-data-full/{unit_id}/{location_id}"

    latest_data = requests.post(get_latest_data_url, headers=headers)

    return latest_data.json()

In [ ]:
rows = last_data_api_extraction()

In [ ]:
# Crear el motor y ejecutar
stage_engine = create_engine(STAGE_DATABASE_URL)

In [ ]:
with stage_engine.connect() as connection:
    response = connection.execute(text("SELECT * FROM stage.landing_scada_data;"))

#response.fetchall()

In [ ]:
with stage_engine.connect() as connection:
    response = connection.execute(text("DELETE FROM stage.landing_scada_data;"))
    connection.commit()

In [ ]:
with open("src/sql/process/V1__raw_to_stage_1.sql", "r") as file:
    raw_to_stage = file.read()

In [ ]:
with stage_engine.connect() as connection:
    connection.execute(text(raw_to_stage), rows)
    connection.commit()

In [ ]:
with stage_engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM stage.landing_scada_data;"))
    rows = result.fetchall()

In [ ]:
len(rows)